# Case 4 -- Steady Constant Sea Level (from Joe)

Comparison with MODFLOW-USG results
Uses Newton formulation

In [ ]:
import pathlib as pl
import numpy as np
import flopy

# path to mf6 shared library
# executables based on mf6 feat-swi branch at: 
#   https://github.com/langevin-usgs/modflow6/tree/feat-swi-correction

# Put the name of the mf6 executable into mf6exe.txt,
# which is not under version control.
with open(pl.Path("./mf6exe.txt"), "r") as f:
    mf6exe = f.readline().strip()
print(f"using executable: {mf6exe}")

sim_ws = pl.Path("./temp/case4")

## MODFLOW-USG results for problem

In [ ]:
hds_usg = np.array(
      [2.42414664e-16, 3.44822330e-01, 5.49943267e-01, 7.01957249e-01,
       8.24049181e-01, 9.26102851e-01, 1.01335249e+00, 1.08894584e+00,
       1.15494018e+00, 1.21276031e+00, 1.26343516e+00, 1.30773084e+00,
       1.34623039e+00, 1.37938411e+00, 1.40754251e+00, 1.43097863e+00,
       1.44990335e+00, 1.46447612e+00, 1.47481228e+00, 1.48098812e+00,
       1.48304388e+00, 1.48098812e+00, 1.47481228e+00, 1.46447612e+00,
       1.44990335e+00, 1.43097863e+00, 1.40754251e+00, 1.37938411e+00,
       1.34623039e+00, 1.30773084e+00, 1.26343516e+00, 1.21276031e+00,
       1.15494018e+00, 1.08894584e+00, 1.01335249e+00, 9.26102851e-01,
       8.24049181e-01, 7.01957249e-01, 5.49943267e-01, 3.44822330e-01,
       0.00000000e+00]
                  )

In [ ]:
zeta_usg = np.array(
    [-9.69658656e-15, -1.37928932e+01, -2.19977307e+01, -2.80782900e+01,
       -3.29619672e+01, -3.70441140e+01, -4.05340994e+01, -4.35578338e+01,
       -4.61976073e+01, -4.85104122e+01, -5.05374063e+01, -5.23092336e+01,
       -5.38492157e+01, -5.51753643e+01, -5.63017003e+01, -5.72391451e+01,
       -5.79961342e+01, -5.85790447e+01, -5.89924913e+01, -5.92395247e+01,
       -5.93217551e+01, -5.92395247e+01, -5.89924913e+01, -5.85790447e+01,
       -5.79961342e+01, -5.72391451e+01, -5.63017003e+01, -5.51753643e+01,
       -5.38492157e+01, -5.23092336e+01, -5.05374063e+01, -4.85104122e+01,
       -4.61976073e+01, -4.35578338e+01, -4.05340994e+01, -3.70441140e+01,
       -3.29619672e+01, -2.80782900e+01, -2.19977307e+01, -1.37928932e+01,
        0.00000000e+00]
)

In [ ]:
hds_usg.shape

## Create and Run Simple Test Model

In [ ]:
#create simple test model
Lx = 2050.0 # meters
ncol = 41
nlay = 1
nrow = 1
delr, delc = Lx / ncol, 1.0
top = 5.
botm = -80.
recharge = 1e-3
k = 10.
h0 = 0.0  
h1 = h0
icelltype = 1
newton_option = "newton"
if newton_option is None:
    linear_accel = "cg"
    max_outer = 50
else:
    linear_accel = "bicgstab"
    max_outer = 500

name = "usg_nr"
sim = flopy.mf6.MFSimulation(
    sim_name=name, 
    sim_ws=sim_ws, 
    exe_name=mf6exe,
    memory_print_option="all"
)
tdis = flopy.mf6.ModflowTdis(sim)
ims = flopy.mf6.ModflowIms(sim, outer_dvclose=1e-8, inner_dvclose=1e-9, linear_acceleration=linear_accel, outer_maximum=max_outer)
gwf = flopy.mf6.ModflowGwf(sim, modelname=name, save_flows=True, newtonoptions=newton_option)
dis = flopy.mf6.ModflowGwfdis(
    gwf, 
    nlay=nlay,
    nrow=nrow,
    ncol=ncol,
    delr=delr,
    delc=delc,
    top=top,
    botm=botm,
)
ic = flopy.mf6.ModflowGwfic(gwf)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    save_specific_discharge=True,
    save_saturation=True,
    # alternative_cell_averaging="AMT-HMK",
    icelltype=icelltype,
    k=k,
)
zeta_file = name + '.zta'
swi = flopy.mf6.ModflowGwfswi(gwf, zeta_filerecord=zeta_file, zetastrt=botm)
chd_spd = [
    ((0, 0, 0), h0),
    ((0, 0, ncol - 1), h1),
]
chd = flopy.mf6.ModflowGwfchd(gwf, stress_period_data=chd_spd)
rch = flopy.mf6.ModflowGwfrcha(gwf, recharge=recharge)
budget_file = name + '.bud'
head_file = name + '.hds'
oc = flopy.mf6.ModflowGwfoc(
        gwf,
        budget_filerecord=budget_file,
        head_filerecord=head_file,
        saverecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
        printrecord=[('HEAD', 'ALL'), ('BUDGET', 'ALL')],
)
sim.write_simulation()
sim.run_simulation()

In [ ]:
fpth = sim_ws / f"{gwf.name}.zta"
zeta_all = flopy.utils.HeadFile(fpth, text="zeta").get_alldata()
zeta = zeta_all[-1].flatten()

x = gwf.modelgrid.xcellcenters.flatten()
head = gwf.output.head().get_data().flatten()
bud = gwf.output.budget()
#print(head.flatten())
#print(zeta.flatten())

spdis = bud.get_data(text='DATA-SPDIS')[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf)
pxs = flopy.plot.PlotCrossSection(gwf, line={"row":0})
ax = pxs.ax
if ncol < 50:
    pxs.plot_grid()
#pxs.plot_bc("ghb")
ax.plot(x, head, "k-")
ax.plot(x, zeta, "k--")
# freshwater
ax.fill_between(x, head, zeta, color="cyan")
# saltwater
ax.fill_between(x, zeta, botm, color="red")
# pxs.plot_vector(qx, qy, qz, normalize=True, color="black")

ax.plot(x, hds_usg, lw=0, mec="black", mfc="none", marker="o")
ax.plot(x, zeta_usg, lw=0, mec="black", mfc="none", marker="x");


## Plot Results

In [ ]:
fpth = sim_ws / f"{name}.dis.grb"
grb = flopy.mf6.utils.MfGrdFile(fpth)
ia = grb.ia
ja = grb.ja
nodes = grb.nodes
flowja = bud.get_data(text="FLOW-JA-FACE")[0].flatten()

# print flows bewteen cells
# for n in range(nodes):
#     for ipos in range(ia[n] + 1, ia[n + 1]):
#         m = ja[ipos]
#         q = flowja[ipos]
#         if m > n:
#             print(f"{n=} {m=} {q=}")

In [ ]:
bobj = gwf.output.budget()

In [ ]:
bobj.get_unique_record_names()

In [ ]:
bobj.get_data(text="DATA-SAT")

In [ ]:
-head * 40.

In [ ]:
zeta

In [ ]:
(-head * 40.) - zeta

In [ ]:
head.max()